# Libraries

In [ ]:
import flirimageextractor
from matplotlib import cm
import os
import matplotlib.pyplot as plt
import numpy as np
import warnings
import tensorflow as tf
import random
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Functions

In [ ]:
class preprocess():
    def __init__(self):
        self.model = flirimageextractor.FlirImageExtractor(palettes=[cm.jet, cm.bwr, cm.gist_ncar])
        
    def extract_thermal_image(self, img_path):
        self.model.process_image(img_path)
        return self.model.get_thermal_np()

def create_cnn(width=140, height=140, depth=1, filters=(16, 32, 64), regress=False):
    inputShape = (height, width, depth)
    chanDim = -1

    inputs = tf.keras.Input(shape=inputShape)
	    # loop over the number of filters
    for (i, f) in enumerate(filters):
            # if this is the first CONV layer then set the input
            # appropriately
        if i == 0:
            x = inputs
            # CONV => RELU => BN => POOL
        x = tf.keras.layers.Conv2D(f, (3, 3), padding="same")(x)
        x = tf.keras.layers.Activation("relu")(x)
        x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
        x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(16)(x)
    x = tf.keras.layers.Activation("relu")(x)
    x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
    x = tf.keras.layers.Dropout(0.5)(x)
        # apply another FC layer, this one to match the number of nodes
        # coming out of the MLP
    x = tf.keras.layers.Dense(4)(x)
    x = tf.keras.layers.Activation("relu")(x)
        # check to see if the regression node should be added
    if regress:
        x = tf.keras.layers.Dense(1, activation="linear")(x)
    # construct the CNN
    model = tf.keras.Model(inputs, x)
    # return the CNN
    return model     

# Preprocessiong

In [ ]:
base_path = 'dataset/230209_IR directory_2.8 V/'
images_path = base_path+'images/'
prep = preprocess()
labels = []

for file in os.listdir(images_path):
    fmt = file.split('.')

    if fmt[-1] == "jpg":
        id = fmt[0]
        labels.append(id.split('_')[-1])
        img_path = images_path+file
        thermal_img = prep.extract_thermal_image(img_path)
        plt.imshow(thermal_img)
        data_path = base_path +'thermal_data/' + id + '.npy'
        np.save(data_path, thermal_img)

labels.sort()      

# Data Preparation

In [ ]:
base_path = 'dataset/230209_IR directory_2.8 V/'
idx = set(range(16))
train_idx =  set(random.sample(idx, 10))
test_idx = idx - train_idx
Y_train = Y[list(train_idx)]
Y_test = Y[list(test_idx)]
X_train = []
X_test = []

for i in train_idx:
    X_train.append(np.load(base_path + 'thermal_data/IR_' + labels[i]+'.npy'))

for i in test_idx:
    X_test.append(np.load(base_path + 'thermal_data/IR_' + labels[i]+'.npy'))

X_train = np.array(X_train)/100
X_test = np.array(X_test)/100

In [ ]:
def create_cnn(width=144, height=144, depth=1, filters=(16, 32, 64)):
    inputShape = (height, width, depth)
    chanDim = -1

    inputs = tf.keras.Input(shape=inputShape)
	    # loop over the number of filters
    for (i, f) in enumerate(filters):
        if i == 0:
            x = inputs
            # CONV => RELU => BN => POOL
        x = tf.keras.layers.Conv2D(f, (3, 3), padding="same")(x)
        x = tf.keras.layers.Activation("relu")(x)
        x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
        x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(128)(x)
    x = tf.keras.layers.Activation("relu")(x)
    x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(64)(x)
    x = tf.keras.layers.Activation("relu")(x)
    x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(16)(x)
    x = tf.keras.layers.Activation("relu")(x)
    x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(11)(x)
    x = tf.keras.layers.Activation("softmax")(x)
    # construct the CNN
    model = tf.keras.Model(inputs, x)
    # return the CNN
    return model  

model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer='adam')

model.fit(X_train, Y_train, epochs=1000, batch_size = 150, verbose=1, validation_split=0.2)

test_results = model.evaluate(X_test, Y_test, verbose=1)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]}%')

In [9]:
model = create_cnn(140, 140, 1, regress=True)
opt = tf.keras.optimizers.Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)
# train the model
print("[INFO] training model...")
model.fit(x=X_train, y=Y_train, 
    validation_data=(X_test, Y_test),
    epochs=1000, batch_size=150)

preds = model.predict(X_test)

1/1 [==============================] - 0s 49ms/step - loss: 69.1116 - val_loss: nan
Epoch 987/1000
1/1 [==============================] - 0s 53ms/step - loss: 71.2913 - val_loss: nan
Epoch 988/1000
1/1 [==============================] - 0s 49ms/step - loss: 74.5047 - val_loss: nan
Epoch 989/1000
1/1 [==============================] - 0s 50ms/step - loss: 68.9283 - val_loss: nan
Epoch 990/1000
1/1 [==============================] - 0s 46ms/step - loss: 70.6451 - val_loss: nan
Epoch 991/1000
1/1 [==============================] - 0s 54ms/step - loss: 73.7369 - val_loss: nan
Epoch 992/1000
1/1 [==============================] - 0s 55ms/step - loss: 77.5239 - val_loss: nan
Epoch 993/1000
1/1 [==============================] - 0s 49ms/step - loss: 71.9024 - val_loss: nan
Epoch 994/1000
1/1 [==============================] - 0s 48ms/step - loss: 67.7643 - val_loss: nan
Epoch 995/1000
1/1 [==============================] - 0s 47ms/step - loss: 72.0187 - val_loss: nan
Epoch 996/1000
1/1 [=====

In [10]:
print("Predicted :", preds.flatten())
print("Actual :", Y_test)

Predicted : [nan nan nan nan nan nan]
Actual : [31.25 37.5  43.75 50.   56.25 75.  ]
